# Analysing Data in tables

In [ ]:
`churn_data` table:

Column number | Column name       | Type      | Description
 ------------ | :---------:           | :---------:           | ------------:
0             | `account_length`        | numerical (`int`)     | Unknown units of time, how long the customer has been a customer.
1             | `international_plan`    | categorical (nominal) | Contract with special conditions for cheaper calls to other countries.
2             | `voice_mail_plan`       | categorical (nominal) | Contract with special conditions for more voicemail storage.
3             | `number_vmail_messages` | numerical (`int`)     | Number of voicemail messages.
4             | `total_day_minutes`     | numerical (`float`)   | Duration in minutes of all calls from 8am to 4pm.
5             | `total_day_calls`      | numerical (`int`)     | Number of all calls from 8am to 4pm.
6             | `total_day_charge`     | numerical (`float`)   | Calculated costs for all calls from 8am to 4pm.
7             | `total_eve_minutes`     | numerical (`float`)   | Duration in minutes of all calls from 4pm to 10pm.
8             | `total_eve_calls`      | numerical (`int`)     | Number of all calls from 4pm to 10pm.
9             | `total_eve_charge`     | numerical (`float`)   | Calculated costs for all calls from 4pm to 10pm.
10             | `total_night_minutes`     | numerical (`float`)   | Duration in minutes of all calls from 10pm to 8am.
11             | `total_night_calls`      | numerical (`int`)     | Number of all calls from 10pm to 8am.
12             | `total_night_charge`     | numerical (`float`)   | Calculated costs for all calls from 10pm to 8am.
13            | `customer_service_calls`| numerical (`int`)     | Number of calls to customer service, e.g. due to technical problems.
14            | `churn`                 | categorical (nominal) | Did the customer leave? (1=yes 0=no)
15            | `local_area_code`       | categorical (nominal) | local area code for telephone.
16            | `phone_num`             | Categorical (nominal) | Customers telephone number not including the local area code.

`cities` table:

Column number | Column name       | Type           | Description
 ------------ | :---------:          | :---------:           | ------------:
0            | `city`       | categorical (nominal) | Cities.
1            | `area_code`       | categorical (nominal) | local area code for telephone.

# 1) Importing Data

In [ ]:
import sqlalchemy as sa
import pandas as pd

In [ ]:
# get access to the tables
engine = sa.create_engine('sqlite:///telco_churn.db')
connection = engine.connect()

In [ ]:
# inspect tables
inspector =sa.inspect(engine)
tables = inspector.get_table_names()
tables

In [ ]:
# read datas in data frame 
url = '''
SELECT *
FROM churn_data
JOIN cities
ON churn_data.local_area_code = cities.area_code
'''

In [ ]:
df = pd.read_sql(url, connection)
df.head()

In [ ]:
# close connection
connection.close()

In [ ]:
# delete unnecessary column
df = df.drop('local_area_code', axis = 1)
df.head()

# **2)** Check and clean the data

In [ ]:
# look at the data
print(df.shape)
df.describe()

In [ ]:
import numpy as np

In [ ]:
# delete rows with inappropriate datas
mask_cust_service = (df.loc[:, 'customer_service_calls'])<0
df.loc[mask_cust_service, :] =np.nan
df = df.dropna(axis = 0, thresh = 18)
df.shape

In [ ]:
# change types to save memory 
float_data = ['account_length', 'number_vmail_messages', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 'total_eve_minutes',
             'total_eve_calls', 'total_eve_charge', 'total_night_minutes', 'total_night_calls', 'total_night_calls', 'total_night_charge',
             'total_night_minutes', 'total_night_calls', 'total_night_charge', 'customer_service_calls', 'phone_num']
fot i in float_data:
    df.loc[:, i] = df.loc[:, i].astype('float32')

category_data = ['international_plan', 'voice_mail_plan', 'city']
for i in category_data:
    df.loc[:, i] = df.loc[:, i].astype('category')

int_data = ['area_code', 'churn']
for i in int_data:
    df.loc[:, i] = df.loc[:, i].astype('int32')

df.dtypes

In [ ]:
# cheack the missing values
df.isna().sum()

# **3)** What are the names of the **four cities** with the highest rates of customer churn?

In [ ]:
# looking for cities with the highest rates of customer churn
cities = pd.crosstab(index = df.loc[:, 'city'], columns = df.loc[:, 'churn']).sort_values(by = 1, ascending=False)
cities = cities[:4]
cities

# **4a)** Checking **categorical** data series for identifying customers who will possibly leave soon

In [ ]:
# check 'international_plan'
intern_plan_crosstab = pd.crosstab(columns = df.loc[:, 'churn'], 
                                    index= df.loc[:, 'international_plan'], 
                                   normalize = 'index')
intern_plan_crosstab

In [ ]:
# check 'voice_mail_plan'
voice_mail_plan_crosstab = pd.crosstab(columns=df.loc[:, 'churn'], 
                                      index=df.loc[:, 'voice_mail_plan'],
                                       normalize='index')
voice_mail_plan_crosstab

In [ ]:
# select customers who may leave based on categorical data
mask_intern_plan = (df.loc[:, 'churn'] == 0) & (df.loc[:, 'international_plan'] =='yes')
df_customers_could_leave = df.loc[mask_intern_plan, :]

In [ ]:
print(df_customers_could_leave.shape)
df_customers_could_leave.head()

# **4b)** Checking **integer** data series and setting the **threshold**

In [ ]:
# change some types to int
df.loc[:, 'account_length'] = df.loc[:, 'account_length'].astype('int32')
df.loc[:, 'number_vmail_messages'] = df.loc[:, 'number_vmail_messages'].astype('int32')
df.loc[:, 'total_day_calls'] = df.loc[:, 'total_day_calls'].astype('int32')
df.loc[:, 'total_eve_calls'] = df.loc[:, 'total_eve_calls'].astype('int32')
df.loc[:, 'total_night_calls'] = df.loc[:, 'total_night_calls'].astype('int32')
df.loc[:, 'customer_service_calls'] = df.loc[:, 'customer_service_calls'].astype('int32')
df.dtypes

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# check integer columns with boxplots
fig, ax = plt.subplots(nrows = 5, ncols = 1, figsize = [10,20])

df.boxplot(column = 'account_length', by = 'churn', ax=ax[0])
df.boxplot(column = 'number_vmail_messages', by = 'churn', ax=ax[1])
df.boxplot(column = 'total_day_calls', by = 'churn', ax=ax[2])
df.boxplot(column = 'total_eve_calls', by = 'churn', ax=ax[3])
df.boxplot(column = 'total_night_calls', by = 'churn', ax=ax[4])

plt.subplots_adjust(wspace=0, hspace=0.5)

In [ ]:
# check integer columns with histograms
fig, ax = plt.subplots(nrows = 5, ncols = 1, figsize = [8,20])

df.groupby('churn')['account_length'].plot(kind = 'hist', ax=ax[0], legend = True, bins=50)
df.groupby('churn')['number_vmail_messages'].plot(kind = 'hist', ax=ax[1], legend = True, bins=50)
df.groupby('churn')['total_day_calls'].plot(kind = 'hist', ax=ax[2], legend = True, bins=50)
df.groupby('churn')['total_eve_calls'].plot(kind = 'hist', ax=ax[3], legend = True, bins=50)
df.groupby('churn')['total_night_calls'].plot(kind = 'hist', ax=ax[4], legend = True, bins=50);

In [ ]:
# select customers based on integer column
customer_service_calls_crosstab = pd.crosstab(columns = df.loc[:, 'churn'], 
                                    index= df.loc[:, 'customer_service_calls'],
                                             normalize = 'index')
mask =(df.loc[:, 'customer_service_calls'] >3) & (df.loc[:, 'churn'] ==0)
df_customers_could_leave2 = df.loc[mask,:]
df_customers_could_leave2.shape

# **4c)** Checking **floating point** data series and determining the threshold by using **logistic regression**

In [ ]:
df.dtypes

In [ ]:
# Check charge and minutes columns
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize = [15,5])
df.plot(kind = 'scatter', 
        x='total_day_minutes', 
        y='total_day_charge', 
        ax=ax[0],
        xlim=(0,500),
        ylim=(0,60));  # check day columns
df.plot(kind='scatter', 
        x='total_eve_minutes', 
        y='total_eve_charge', 
        ax=ax[1],
        xlim=(0,500),
        ylim=(0,60));  # check evening columns
df.plot(kind='scatter', 
        x='total_night_minutes', 
        y='total_night_charge', 
        ax=ax[2],
        xlim=(0,500),
        ylim=(0,60)); 

In [ ]:
# drop outlines
mask_total_day = df.loc[:, 'total_day_minutes']>400
print(df.shape)
df = df.drop(df.index[mask_total_day], axis=0)
df.shape

In [ ]:
# check other float columns with boxplots and histograms
fig, ax = plt.subplots(nrows = 6, ncols = 2, figsize = [15,25])

df.groupby('churn')['total_day_minutes'].plot(kind = 'hist', ax=ax[0][0], legend = True, bins=50)
df.groupby('churn')['total_day_charge'].plot(kind = 'hist', ax=ax[1][0], legend = True, bins=50)
df.groupby('churn')['total_eve_minutes'].plot(kind = 'hist', ax=ax[2][0], legend = True, bins=50)
df.groupby('churn')['total_eve_charge'].plot(kind = 'hist', ax=ax[3][0], legend = True, bins=50)
df.groupby('churn')['total_night_minutes'].plot(kind = 'hist', ax=ax[4][0], legend = True, bins=50)
df.groupby('churn')['total_night_charge'].plot(kind = 'hist', ax=ax[5][0], legend = True, bins=50)

df.boxplot(column = 'total_day_minutes', by = 'churn', ax=ax[0][1])
df.boxplot(column = 'total_day_charge', by = 'churn', ax=ax[1][1])
df.boxplot(column = 'total_eve_minutes', by = 'churn', ax=ax[2][1])
df.boxplot(column = 'total_eve_charge', by = 'churn', ax=ax[3][1])
df.boxplot(column = 'total_night_minutes', by = 'churn', ax=ax[4][1])
df.boxplot(column = 'total_night_charge', by = 'churn', ax=ax[5][1])

plt.subplots_adjust(wspace=0.2, hspace=0.5)

In [ ]:
import seaborn as sns

In [ ]:
# pair plot
df_new = df.loc[:, ['total_day_minutes','total_day_charge','total_eve_minutes','total_eve_charge','total_night_minutes',
                    'churn', 'total_night_charge']]
sns.pairplot(df_new)

In [ ]:
# correlation matrix
sns.heatmap(df_new.corr(), annot=True);

In [ ]:
# logistic model
import statsmodels.formula.api as smf

In [ ]:
# define a logit model and fit it
model = smf.logit(formula='churn ~ total_day_minutes', data=df_new)
results = model.fit()

In [ ]:
# check summary
results.summary()

In [ ]:
# Use the trained results to predict a given range 
X = pd.Series(range(800))  
X_df = pd.DataFrame(X)
X_df.columns = ['total_day_minutes']
X_df.head()

In [ ]:
# plot the S-curve
p_y = results.predict(X_df)
p_y.plot()

In [ ]:
# Calculate treshold value
p_y[p_y>=0.5].index[0]

In [ ]:
# plot treshold value as a vertical line in logistic regression plot
fig, ax = plt.subplots()
p_y.plot()
ax.vlines(x=p_y[p_y>=0.5].index[0], ymin=0, ymax=1)

In [ ]:
# identify customer(s) to be contacted based on critical value from logistic regression
mask = df_new.loc[:, 'total_day_minutes']>p_y[p_y>=0.5].index[0]
df.loc[mask,:]

# **5)** Visualizing the cities and other selected data series.

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
# column chart of urban areas
fig, ax = plt.subplots(figsize = [10,5])

color = ['#C154C1','#702963']

cities = pd.crosstab(index = df.loc[:, 'city'], columns = df.loc[:, 'churn']).sort_values(by = 1, ascending=False)

cities.plot(kind = 'bar', ax=ax, title = 'Cities with the highest rates of customer churn', legend = ['yes', 'no'],
           color = color, ylim = [0,340])

ax.set_ylabel('Number of people', fontsize=13)
ax.set_xlabel('City', fontsize=15)

ax.legend(['Current customers', 'Customers who left'],
          fancybox=True, 
          framealpha=1, 
          shadow=True, 
          borderpad=0.7,  
          loc='upper right', 
          bbox_to_anchor=(1.1, 1.01))

ann = ax.annotate(s='Cities to target with poster campaigns',
                  xy=[0, 240],
                  xytext=[0, 285],
                  arrowprops=dict(facecolor='#8B008B'))
ann1 = ax.annotate(s='',
                  xy=[1, 240],
                  xytext=[1, 280],
                  arrowprops=dict(facecolor='#8B008B'))
ann2 = ax.annotate(s='',
                  xy=[2, 240],
                  xytext=[2, 280],
                  arrowprops=dict(facecolor='#8B008B'))
ann3 = ax.annotate(s='',
                  xy=[3, 245],
                  xytext=[3, 280],
                  arrowprops=dict(facecolor='#8B008B'))

ax.xaxis.set_tick_params(labelrotation=45);

In [ ]:
# chart of categorical data
intern_plan_crosstab = pd.crosstab(index = df.loc[:, 'churn'], 
                                   columns = df.loc[:, 'international_plan'], normalize = 'columns')

fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = [10,5])

color = ['#702963', '#C154C1']

intern_plan_crosstab.plot(kind = 'pie', y = 0, ax=ax[0], legend = False, labels =['', ''],
                          explode = (0, 0.15), colors=color)
intern_plan_crosstab.plot(kind = 'pie', y = 1, ax=ax[1], legend = True, labels =['', ''],  
                          explode = (0, 0.08),colors=color)

ax[1].legend(['Current customers', 'Customers who left'],
          fancybox=True, 
          framealpha=1, 
          shadow=True, 
          borderpad=0.5,  
          loc='upper right', 
          bbox_to_anchor=(1.1, 1.09),
            fontsize=12)
fig.suptitle('Percentatge of customers who use international plan')

ann = ax[1].annotate(s='',
                  xy=[0.4, 0],
                  xytext=[0.3, -0.4],
                  arrowprops=dict(facecolor='#8B008B'))
text = ax[1].text(x=-0.4,
           y=-0.5,
           s='Customers who may leave')

ax[0].set(xlabel = 'Without international plan', ylabel ='')
ax[1].set(xlabel= 'Have international plan', ylabel='');

In [ ]:
# chart of integer data
customer_service_calls_crosstab = pd.crosstab(columns = df.loc[:, 'churn'], 
                                    index= df.loc[:, 'customer_service_calls'], normalize = 'index')

fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = [13,5])

color = ['#C154C1','#702963']

customer_service_calls_crosstab.plot(kind = 'bar', ax=ax[0], legend = False, color=color)
customer_service_calls_crosstab.loc[4:8].plot(kind = 'bar', ax=ax[1], color=color, ylim = [0,1])

fig.suptitle('Percentatge of customers who made service calls')
ax[0].set_ylabel('Percent of people', fontsize = 13) 
ax[0].set_xlabel('Service calls', fontsize = 13)
ax[1].set_xlabel('Service calls', fontsize = 13)
ax[1].set_yticklabels([None])

ax[1].legend(['Current customers', 'Customers who left'],
          fancybox=True, 
          framealpha=1, 
          shadow=True, 
          borderpad=0.5,  
          loc='upper right', 
            fontsize=12)
text = ax[1].text(x=-0.3,
           y=0.75,
           s='Current customers who are probably ready to leave')

ann = ax[1].annotate(s='',
                  xy=[0, 0.6],
                  xytext=[0, 0.7],
                  arrowprops=dict(facecolor='#8B008B'))
ann = ax[1].annotate(s='',
                  xy=[1, 0.6],
                  xytext=[1, 0.7],
                  arrowprops=dict(facecolor='#8B008B'))
ann = ax[1].annotate(s='',
                  xy=[2, 0.635],
                  xytext=[2, 0.7],
                  arrowprops=dict(facecolor='#8B008B'))
ann = ax[1].annotate(s='',
                  xy=[3, 0.6],
                  xytext=[3, 0.7],
                  arrowprops=dict(facecolor='#8B008B'))
ann = ax[1].annotate(s='',
                  xy=[4, 0.6],
                  xytext=[4, 0.7],
                  arrowprops=dict(facecolor='#8B008B'))


plt.subplots_adjust(wspace=0.05, hspace=0.5);

In [ ]:
# plot for findings in logistic regression
fig, ax = plt.subplots(figsize = [8,5])

p_y.plot(ax=ax, color='#C154C1', xlim = [0,750])

plt.ylabel('Probability of churn', fontsize = 13)
plt.xlabel('Minutes pro day', fontsize = 13)


ax.axvline(p_y[p_y>=0.5].index[0], color='#B284BE')

ann = ax.annotate(s='The threshold value',
                  xy=[345, 0.5],
                  xytext=[100, 0.62],
                  arrowprops=dict(facecolor='#8B008B'));


# **6)** Formulating a recommendation

In [ ]:
# observations, notes and recommendations 

# Firstly, the cities with the highest customer churn are Jacksonville,Orlando1, Cape Coral, and Orlando2. 
# They must be targeted with poster campaigns.

# For the second marketing campaign are a few cases.

# Among people who used the international plan, 42% left. 
# Therefore, it is necessary to approach individuals with the remaining 58%. It is 182 people.

# Сompany should pay attention to people who often called the service 
# because many people left who made more than three calls. 129 people made a lot of calls and could potentially leave.

# The logistic regression showed only one person, and he no longer uses the services of your company.

# Overall you should approach individually of 311 people.